In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:





dataset=pd.read_csv('../data/AMAZON_FASHION.csv',names=['index','pID','image','ratings','reviewtext','time','cust_id','cust_name','style','summary','unixtime','verified','vote'],low_memory=False)

In [3]:
dataset.head()

index         pID  image  ratings  \
0    NaN        asin  image  overall   
1    0.0  7106116521    NaN        5   
2    1.0  7106116521    NaN        2   
3    2.0  7106116521    NaN        4   
4    3.0  7106116521    NaN        2   

                                          reviewtext         time  \
0                                         reviewText   reviewTime   
1                             Exactly what I needed.  10 20, 2014   
2  I agree with the other review, the opening is ...  09 28, 2014   
3  Love these... I am going to order another pack...  08 25, 2014   
4                                too tiny an opening  08 24, 2014   

          cust_id     cust_name  style  \
0      reviewerID  reviewerName  style   
1  A1D4G1SNUZWQOT         Tracy    NaN   
2  A3DDWDH9PX2YX2     Sonja Lau    NaN   
3  A2MWC41EW7XL15      Kathleen    NaN   
4  A2UH2QQ275NV45   Jodi Stoner    NaN   

                                             summary        unixtime  \
0                                            summary  unixReviewTime   
1                             perfect replacements!!      1413763200   
2  I agree with the other review, the opening is ...      1411862400   
3                                My New 'Friends' !!      1408924800   
4                                          Two Stars      1408838400   

   verified  vote  
0  verified  vote  
1      True   NaN  
2      True   3.0  
3     False   NaN  
4      True   NaN

In [4]:
# building the environment with data 
# this will give positive reinforcements for the agent to learn
# using Actor crtic approach
# temporal difference
# q learning
# sarsa
# deep sarsa
# actor2Crtic

dataset.ratings.unique()


array(['overall', '5', '2', '4', '3', '1'], dtype=object)

In [5]:
len(dataset.cust_id.unique())

749234

In [6]:
len(dataset)

883637

In [7]:
len(dataset.pID.unique())

186190

In [8]:
# lets understand the data first


In [9]:
dataset=dataset.drop(['index','image','time','cust_name','style','reviewtext','unixtime'],axis=1)[1:]

In [65]:
dataset[dataset.ratings=='overall']

Empty DataFrame
Columns: [pID, ratings, cust_id, summary, verified, vote]
Index: []

In [11]:
dataset.vote.isnull().sum()

803736

In [12]:
803736/883637

0.9095771227325248

In [13]:
# 10 % data in vote is not present
dataset.info()
import numpy as np

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883636 entries, 1 to 883636
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   pID       883636 non-null  object
 1   ratings   883636 non-null  object
 2   cust_id   883636 non-null  object
 3   summary   883099 non-null  object
 4   verified  883636 non-null  object
 5   vote      79900 non-null   object
dtypes: object(6)
memory usage: 40.4+ MB


In [14]:
# we are going to embed the pID(product ids) baed on ratings 
# getting the dataset for embedding the profuct id
# we will be embedding the product id s in latent space 128 dimensions

In [15]:
embedding_dataset=dataset.drop(['cust_id','verified','vote'],axis=1)

In [16]:
# embedding_dataset.head()
# l=[]
# embedding_dataset['summary'].apply(lambda review:[l.append(x) for x in review.split()])

In [76]:
Xtrain,ytrain=embedding_dataset.drop(['ratings'],axis=1),embedding_dataset['ratings'].apply(lambda x:str(int(x)-1)).astype(np.int64)

In [77]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [78]:
Xtrain.summary=Xtrain.summary.fillna('')

In [79]:
product_embeds=TextVectorization(max_tokens=186191,output_sequence_length=1)

In [80]:
# summary_embeds=TextVectorization(max_tokens=6000,output_sequence_length=30)
product_embeds.adapt(Xtrain.pID.values)

In [81]:
# summary_embeds.adapt(Xtrain.summary)

In [82]:
product_embeds(Xtrain.tail()['pID'].values)

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[15225],
       [15225],
       [15225],
       [86229],
       [86234]], dtype=int64)>

In [83]:
Xtrain.pID.unique()

array(['7106116521', 'B00007GDFV', 'B00008JOQI', ..., 'B01HJHTH5U',
       'B01HJHF97K', 'B01HJG5NMW'], dtype=object)

In [84]:
# product embeds completed

In [85]:
product_input=tf.keras.Input(shape=(),dtype=tf.string)
te=product_embeds(product_input)
embedding=tf.keras.layers.Embedding(input_dim=186191,output_dim=128, input_length=1)(te)
sq=tf.keras.backend.squeeze(embedding,1)
model1=tf.keras.Model(product_input,sq)

In [86]:
model1(np.array(Xtrain.pID.values[10:12]))

<tf.Tensor: shape=(2, 128), dtype=float32, numpy=
array([[-0.02045945, -0.02673645,  0.01228192, -0.03772694,  0.01484864,
        -0.02793639,  0.00593524, -0.03337293, -0.00535931,  0.04430277,
         0.01595685,  0.03778611, -0.04806355, -0.0156961 ,  0.04769994,
        -0.03791586,  0.04056395, -0.0234091 ,  0.00010987,  0.03270366,
        -0.02969217, -0.00556561, -0.04716283, -0.02429191, -0.02153492,
        -0.02391226, -0.0407507 ,  0.0489707 ,  0.03712385,  0.02976793,
         0.04452189, -0.0368383 ,  0.03179942, -0.04007597,  0.04219092,
         0.0297763 , -0.01626092,  0.02876313,  0.00211476,  0.03446623,
         0.01449784, -0.01319506,  0.00842694, -0.0216571 , -0.00227325,
        -0.0119347 ,  0.02302844, -0.00382463, -0.04126521, -0.02484237,
        -0.00770999, -0.002448  , -0.01273403,  0.00075687,  0.03540519,
        -0.03145092,  0.04866964,  0.04302968,  0.01969062,  0.03322807,
        -0.02282285, -0.04986739,  0.03733969,  0.02136943,  0.03866594,
 

In [87]:
import tensorflow_hub as hub

embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",trainable=False,input_shape=[])
embeddings = embed(["product was n"])

# this willl take more time to load as it is 1 gb module loaded
print(embeddings)
# embL=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",trainable=False,input_shape=[])

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


tf.Tensor(
[[ 4.74630669e-02  3.29148993e-02  8.56172200e-03  2.75905095e-02
  -9.00743995e-03  2.94360351e-02  8.63117129e-02  5.79298567e-03
   7.96186253e-02  2.26764940e-03  3.94520909e-03  5.85150346e-02
   6.28314540e-02  1.04013793e-02  1.37615297e-03 -1.72595903e-02
   7.60914758e-02  2.74546747e-03  4.81688716e-02  2.38764379e-03
   4.03652638e-02 -5.00830412e-02  2.63309516e-02  6.96642837e-03
  -2.72083692e-02  1.79889631e-02 -4.68126126e-03  3.65154035e-02
  -5.57890395e-03 -3.75934020e-02 -2.98452731e-02  8.12000223e-03
  -8.08521919e-03  2.96422243e-02  1.09191239e-03 -1.55707775e-02
   3.02325878e-02 -1.71842761e-02  2.84885243e-02 -2.65493318e-02
   1.60361975e-04  5.91537915e-02  3.20559666e-02  5.69830276e-02
   2.17608046e-02 -3.36717740e-02 -9.89561714e-03 -5.49412966e-02
   4.44501154e-02  3.69915627e-02  2.50509623e-02  8.46845796e-04
  -5.67864403e-02  6.98083565e-02  3.89808714e-02 -5.34078591e-02
   6.88946471e-02 -1.58286635e-02  3.35768014e-02  4.98532504e-02

In [88]:
summary_input=tf.keras.Input(shape=(),dtype=tf.string)
use=embed(summary_input)
final=tf.keras.layers.Dense(128,activation='relu')(use)
model2=tf.keras.Model(summary_input,final)



In [89]:
model2(Xtrain.summary.values[1:4])

<tf.Tensor: shape=(3, 128), dtype=float32, numpy=
array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        6.33123219e-02, 1.14690829e-02, 2.16484070e-04, 9.07491669e-02,
        3.03274393e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.42564455e-03, 0.00000000e+00, 7.74736404e-02,
        4.63851616e-02, 0.00000000e+00, 6.74336255e-02, 5.52842990e-02,
        0.00000000e+00, 0.00000000e+00, 1.42701082e-02, 6.85986876e-03,
        5.98851219e-03, 1.96399018e-02, 9.04401690e-02, 0.00000000e+00,
        1.92954801e-02, 7.59483948e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.18663940e-02, 3.33156101e-02,
        6.84497133e-02, 0.00000000e+00, 0.00000000e+00, 3.41609865e-02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.43319720e-02, 0.00000000e+00, 1.23623610e-02, 5.24963327e-02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        5.8461

In [90]:
model1.outputs


[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'tf.compat.v1.squeeze_2')>]

In [91]:
x=tf.keras.layers.Concatenate()([model1.output,model2.output])

In [92]:
dense_1= tf.keras.layers.Dense(128,activation='relu')(x)
dense_2=tf.keras.layers.Dense(5, activation='softmax')(dense_1) 


In [93]:
final_model=tf.keras.Model(inputs=[product_input,summary_input],outputs=dense_2)

In [94]:
# img_file = './model_arch.png'

# tf.keras.utils.plot_model(final_model, to_file=img_file, show_shapes=True, show_layer_names=True)

In [95]:
from tensorflow.keras.utils import plot_model

plot_model(final_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [96]:
final_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 text_vectorization_1 (TextVect  (None, 1)           0           ['input_5[0][0]']                
 orization)                                                                                       
                                                                                                  
 input_6 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 1, 128)       23832448    ['text_vectorization_1[0][0

In [97]:
data1=tf.data.Dataset.from_tensor_slices(Xtrain.pID.values)
data2=tf.data.Dataset.from_tensor_slices(Xtrain.summary.values)
target=tf.data.Dataset.from_tensor_slices(ytrain.values)
daata=tf.data.Dataset.zip(((data1,data2),target)).batch(32).prefetch(1)

In [98]:
final_model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),)

In [99]:
# # final_model.fit(daata,epochs=1)
# model2(Xtrain.summary.values)

In [100]:
ytrain.unique()

array([4, 1, 3, 2, 0], dtype=int64)

In [101]:
final_model.fit(daata,epochs=1)

 1483/27614 [>.............................] - ETA: 2:13:29 - loss: 0.6524

KeyboardInterrupt: 